<a href="https://colab.research.google.com/github/cedjustin/AI/blob/master/sprint13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a,b)
sess = tf.compat.v1.Session()
output = sess.run(add)
print(output) 
sess.close()

12


In [23]:
with tf.compat.v1.Session() as sess:
  output = sess.run(tf.add(a,b))
  print(output)

12


In [24]:
import numpy as np
a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) 

12


In [25]:
tf = tf.compat.v1
number_holder = tf.placeholder(tf.int32)
number_holder2 = tf.placeholder(tf.int32)
with tf.Session() as sess:
  add = tf.add(number_holder2,number_holder)
  output = sess.run(add, feed_dict={number_holder2: 1,number_holder: 2})
  print(output)
  sess.close()

3


In [26]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [28]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

# weights and bias
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [30]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [31]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [32]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))

In [33]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [44]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoch in range(1000):
    sess.run(train_step, feed_dict={x: x_train, t:y_train})
    if epoch % 100 == 0:
      acc_val = sess.run(accuracy, feed_dict={x:x_train, t: y_train})
      print('epoch: {}, Accuracy: {}'.format(epoch, acc_val))
  classified = sess.run(correct_prediction, feed_dict={x: x_train, t: y_train})
  print(classified)
  prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
  })
  print(prob)
  print(sess.run(W))
  print(sess.run(b))
  sess.close()

epoch: 0, Accuracy: 0.75
epoch: 100, Accuracy: 1.0
epoch: 200, Accuracy: 1.0
epoch: 300, Accuracy: 1.0
epoch: 400, Accuracy: 1.0
epoch: 500, Accuracy: 1.0
epoch: 600, Accuracy: 1.0
epoch: 700, Accuracy: 1.0
epoch: 800, Accuracy: 1.0
epoch: 900, Accuracy: 1.0
[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9648671e-04]
 [4.9049795e-02]
 [4.9049795e-02]
 [9.3120372e-01]]
[[5.569954]
 [5.569954]]
[-8.534579]
